In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
#from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop

In [4]:
data= open('/content/drive/MyDrive/Data science/job_tests/Ai bees/comments_generator_data/001.txt').read()

In [5]:
data = " ".join(data) # is ko se nhi karogy to apni clearning tarikay se bi nhi chalega oor apne tarikay k bina bi hang hoga. to ye line use karna must

In [6]:
import string
def clean_data(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower for word in tokens]
  return tokens

In [7]:
tokens= clean_data(data)
print(tokens[:20])


[<built-in method lower of str object at 0x7fcad15ca8b0>, <built-in method lower of str object at 0x7fcad15f0130>, <built-in method lower of str object at 0x7fcad1643f70>, <built-in method lower of str object at 0x7fcad1641630>, <built-in method lower of str object at 0x7fcad15f0430>, <built-in method lower of str object at 0x7fcad15dd430>, <built-in method lower of str object at 0x7fcad1643f70>, <built-in method lower of str object at 0x7fcad161aaf0>, <built-in method lower of str object at 0x7fcad15f03b0>, <built-in method lower of str object at 0x7fcad15f03b0>, <built-in method lower of str object at 0x7fcad1643f70>, <built-in method lower of str object at 0x7fcad16888f0>, <built-in method lower of str object at 0x7fcad15cadb0>, <built-in method lower of str object at 0x7fcad161f1b0>, <built-in method lower of str object at 0x7fcad17047f0>, <built-in method lower of str object at 0x7fcad15dd430>, <built-in method lower of str object at 0x7fcad15eab70>, <built-in method lower of str 

In [8]:
len(tokens)

len(set(tokens))



43

In [9]:


length = 50 + 1

lines = []
for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join([str(i) for i in seq])
  lines.append(line)
  if i > 200000:
    break
 

In [10]:
print(len(lines))

1931


In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)

sequences = tokenizer.texts_to_sequences(lines)



In [12]:
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]


In [13]:
tokenizer.word_index


{'0x7fcad158aa30': 40,
 '0x7fcad1591930': 28,
 '0x7fcad15919b0': 49,
 '0x7fcad1591ab0': 34,
 '0x7fcad1591af0': 51,
 '0x7fcad1591b30': 47,
 '0x7fcad15ca8b0': 33,
 '0x7fcad15ca8f0': 35,
 '0x7fcad15ca970': 37,
 '0x7fcad15ca9f0': 46,
 '0x7fcad15cadb0': 30,
 '0x7fcad15d16b0': 24,
 '0x7fcad15d1e30': 41,
 '0x7fcad15dd430': 9,
 '0x7fcad15ea7b0': 16,
 '0x7fcad15eaa70': 45,
 '0x7fcad15eaab0': 43,
 '0x7fcad15eaaf0': 42,
 '0x7fcad15eab30': 44,
 '0x7fcad15eab70': 32,
 '0x7fcad15eabb0': 48,
 '0x7fcad15f00b0': 38,
 '0x7fcad15f0130': 20,
 '0x7fcad15f0170': 25,
 '0x7fcad15f01f0': 17,
 '0x7fcad15f03b0': 14,
 '0x7fcad15f0430': 18,
 '0x7fcad1607530': 50,
 '0x7fcad161aaf0': 26,
 '0x7fcad161f1b0': 12,
 '0x7fcad161f330': 39,
 '0x7fcad161f370': 29,
 '0x7fcad163d2b0': 27,
 '0x7fcad1641230': 36,
 '0x7fcad1641630': 15,
 '0x7fcad1643f70': 13,
 '0x7fcad164fc70': 10,
 '0x7fcad1680c70': 21,
 '0x7fcad1688130': 22,
 '0x7fcad16888f0': 11,
 '0x7fcad16a3c70': 19,
 '0x7fcad16b07b0': 31,
 '0x7fcad17047f0': 23,
 'at': 8,
 '

In [14]:
vocab_size = len(tokenizer.word_index) + 1


In [15]:
y= to_categorical(y,num_classes=vocab_size)

In [16]:
seq_length = X.shape[1]

In [17]:
model = Sequential()
model.add(Embedding(vocab_size,50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))

model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 458, 50)           2600      
_________________________________________________________________
lstm (LSTM)                  (None, 458, 100)          60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 52)                5252      
Total params: 158,752
Trainable params: 158,752
Non-trainable params: 0
_________________________________________________________________


In [19]:
# # Defining a helper function to save the model after each epoch
# # in which the loss decreases
# filepath = "/content/drive/MyDrive/Data science/job_tests/Ai bees/comments_generator_data/news_post_weights.pth"
# checkpoint = ModelCheckpoint(filepath, monitor ='loss',
# 							verbose = 1, save_best_only = True,
# 							mode ='min')


In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit(X, y, batch_size = 512, epochs=1)

4/4 [==============================] - 18s 5s/step - loss: 3.9412 - accuracy: 0.0596


In [29]:
def gen_text(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating ='pre')
    y_predic = np.argmax(model.predict_classes(encoded), axis=-1)#model.predict_classes(encoded)
    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predic:
        predicted_word = word
        break
    seed_text = seed_text +''+predicted_word
    text.append(predicted_word)
  return ''.join(text)


In [32]:
gen_text(model, tokenizer, seq_length, "Time Warner's fourth quarter profits were slightly better than analysts' expectation'", 100)

''

In [ ]:
# Save the model in h5 format 
model.save("/content/drive/MyDrive/Data science/job_tests/Ai bees/models/category/comments.h5")